# Outlier Analysis And Anomaly Detection 

## **CONTENT :**

1. [Basic Overview](#1)
2. [wine_benchmark](#2)<br>
2.1.[Method: KNN](#2.1)<br>
2.2.[Method: CBLOF](#2.2)<br>
2.3.[Method: LOF](#2.3)<br>
2.4.[Method: IForest](#2.4)<br>
2.5.[Conclusion Analysis](#2.5)<br>
3. [Skin_benchmark](#2)<br>
3.1.[Method: KNN](#3.1)<br>
3.2.[Method: CBLOF](#3.2)<br>
3.3.[Method: LOF](#3.3)<br>
3.4.[Method: IForest](#3.4)<br>
3.5.[Conclusion Analysis](#3.5)<br>


<a id="1"></a> <br>
> ## **1. Basic Overview**

#### 数据集选择 skin_benchmarks & wine_benchmarks

#### 编程语言：python

首先导入所需各类依赖包：

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # 可视化
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly.tools import FigureFactory as ff

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.utils import column_or_1d
from sklearn.utils import check_consistent_length
from pyod.utils.utility import precision_n_scores
from pyod.utils.data import evaluate_print

from pyod.models.knn import KNN
from pyod.models.cblof import CBLOF
from pyod.models.lof import LOF
from pyod.models.iforest import IForest

from time import *
import sys
import os

import warnings # 忽略warnings
warnings.filterwarnings('ignore')

<a id="2"></a> <br>
> ## **2. wine benchmark**

以wine_benchmark中第一个数据集为例，我们先来看一下数据集中的数据概况：

In [2]:
wineExp = pd.read_csv("./wine/benchmarks/wine_benchmark_0001.csv")

首先让我们先来看看该数据集的头和尾，下表显示了数据集开头的前4行。<br>

In [3]:
d = wineExp.head(5)
colorscale = "YlOrRd"
table = ff.create_table(d, colorscale=colorscale)
for i in range(len(table.layout.annotations)):
    table.layout.annotations[i].font.size = 9
    table.layout.annotations[i].width = 50
    table.layout.annotations[i].font.color = 'black'
iplot(table)

In [4]:
wineExp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3703 entries, 0 to 3702
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   point.id              3703 non-null   object 
 1   motherset             3703 non-null   object 
 2   origin                3703 non-null   object 
 3   original.label        3703 non-null   int64  
 4   diff.score            3703 non-null   float64
 5   ground.truth          3703 non-null   object 
 6   fixed.acidity         3703 non-null   float64
 7   volatile.acidity      3703 non-null   float64
 8   citric.acid           3703 non-null   float64
 9   residual.sugar        3703 non-null   float64
 10  chlorides             3703 non-null   float64
 11  free.sulfur.dioxide   3703 non-null   float64
 12  total.sulfur.dioxide  3703 non-null   float64
 13  density               3703 non-null   float64
 14  pH                    3703 non-null   float64
 15  sulphates            

从上面的例子可以看到，数据集中每一个.csv文件包含多条wine_point的数据记录，同时数据中也给出了wine_point所对应的各种红酒成分数值属性如**acidity, sugar, chlorides, etc.**，为了能够有test集合上进行验证的，我们发现GroundTruth于origin.label提供了每一条记录点是否为异常点的记录，于是将此作为test集合的label。下面我们来分别处理benchmarks中的每一个.csv文件：

In [9]:
wine_bm_path = "./wine/benchmarks"
wine_bm_files = []
for wine_bm_file in os.listdir(wine_bm_path):
    file_path = os.path.join(wine_bm_path, wine_bm_file)
    wine_bm_files.append(file_path)
    
# 打乱benchmarks中的.csv文件顺序    
np.random.shuffle(wine_bm_files)
# 后面算法用来分割训练集与测试集
train_rate = 0.7

length = len(wine_bm_files)
print("=" * 70)
print("The Total Number of Benchmark is: " , length)
print("e.g. : ", wine_bm_files[0])
print("=" * 70)

The Total Number of Benchmark is:  1210
e.g. :  ./wine/benchmarks/wine_benchmark_1343.csv


In [10]:
df = pd.read_csv(wine_bm_files[0])
print("=" * 70)
print("The shape of DataFrame is:", df.shape)
print("=" * 70)

d = df.head()
colorscale = "YlOrRd"
table = ff.create_table(d, colorscale=colorscale)
for i in range(len(table.layout.annotations)):
    table.layout.annotations[i].width = 40
    table.layout.annotations[i].font.size = 9
    table.layout.annotations[i].font.color = 'black'
iplot(table)

The shape of DataFrame is: (3490, 17)


读取数据集中的数据后可以看到，该数据集总共包括1210个benchmark，每个benchmark中记录数不同，在这里我们选取其中的**fixed.acidity** & **volatile.acidity** & **citric.acid** & **residual.sugar** & **chlorides** & **free.sulfur.dioxide** & **total.sulfur.dioxide** & **density** & **pH** & **sulphates** & **alcohol** 十一个属性值，同时选取**ground.truth**值来完成本次离群点分析和异常检测的实验。<br>
其中分析算法我们选择：**KNN** & **CBLOF** & **LOF** & **IForest** 

<a id="2.1"></a> <br>
> ###  **2.1 Method: KNN**

基于**k-近邻平均距离（kNN）的离群点检测算法**是一种比较简单的检测方法，其算法的流程是：输入数据集D，参数k和n；对于每个点计算它的k-近邻距离；按照k-近邻距离进行降序排序，前n个点即可认为是离群点，代码如下所示：

In [11]:
wine_roc_mean = []
wine_rank_n_mean = []
wine_time_total = []

# KNN算法
begin_time = time()
knn_roc = []
knn_rank_n =[]

for i in range(length):
    df = pd.read_csv(wine_bm_files[i])
    # 选取属性特征
    Xattr = df[["fixed.acidity","volatile.acidity","volatile.acidity", "residual.sugar", "chlorides", 
               "free.sulfur.dioxide", "total.sulfur.dioxide", "density", "sulphates", "alcohol"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'KNN'
    clf = KNN(n_neighbors = 5) 
    clf.fit(X_train) # 拟合检测器clf
    
    # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        knn_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        knn_rank_n.append(prn_)
    except ValueError:
        pass
   
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)
    
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
wine_roc_mean.append(np.mean(knn_roc))
wine_rank_n_mean.append(np.mean(knn_rank_n))
wine_time_total.append(run_time)

Current Iter 0
On Training Data:
KNN ROC:0.5578, precision @ rank n:0.0833

On Test Data:
KNN ROC:0.5891, precision @ rank n:0.1091
Current Iter 1000
On Training Data:
KNN ROC:0.5486, precision @ rank n:0.4219

On Test Data:
KNN ROC:0.5438, precision @ rank n:0.4095


In [12]:
# 可视化展示
x_ = [x for x in range(len(knn_roc))]
trace = go.Scatter(x = x_, y = knn_roc, mode = "lines + markers", name = "KNN_ROC",
                   marker = dict(color='rgba(249, 94, 28, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'KNN_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='KNN_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="2.2"></a> <br>
> ###  **2.2 Method: CBLOF**

**CBLOF算法**是基于聚类的方法，基于聚类的方法通过考察对象与簇之间的关系检测离群点。其算法的流程是：找出数据集中的簇，并把它们按大小降序排列；对于每个数据点赋予基于簇的局部离群点因子（CBLOF），对于属于大簇的点，它的CBLOF是簇的大小和该点与簇的相似性的乘积。CBLOF用统计学方法定义点和簇之间的相似性，代表点属于簇的概率，该值越大，点与簇越相似，CBLOF值可以检测远离任何簇的离群点。

In [13]:
# CBLOF算法
begin_time = time()
cblof_roc = []
cblof_rank_n = []

for i in range(length):
    df = pd.read_csv(wine_bm_files[i])
    # 选取属性特征
    Xattr = df[["fixed.acidity","volatile.acidity","volatile.acidity", "residual.sugar", "chlorides", 
               "free.sulfur.dioxide", "total.sulfur.dioxide", "density", "sulphates", "alcohol"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'CBLOF'
    clf = CBLOF(random_state=42) 
    clf.fit(X_train) # 拟合检测器clf
    
     # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        cblof_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        cblof_rank_n.append(prn_)
    except ValueError:
        pass
    
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)

        
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
wine_roc_mean.append(np.mean(cblof_roc))
wine_rank_n_mean.append(np.mean(cblof_rank_n))
wine_time_total.append(run_time)

Current Iter 0
On Training Data:
CBLOF ROC:0.5485, precision @ rank n:0.0866

On Test Data:
CBLOF ROC:0.5056, precision @ rank n:0.0208
Current Iter 1000
On Training Data:
CBLOF ROC:0.5427, precision @ rank n:0.4164

On Test Data:
CBLOF ROC:0.5311, precision @ rank n:0.3892


In [14]:
# 可视化展示
x_ = [x for x in range(len(cblof_roc))]
trace = go.Scatter(x = x_, y = cblof_roc, mode = "lines + markers", name = "CBLOF_ROC",
                   marker = dict(color='rgba(249, 94, 28, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'CBLOF_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='CBLOF_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="2.3"></a> <br>
> ###  **2.3 Method: LOF**

**LOF算法**是一种基于距离的异常检测方法，LOF通过计算一个数值score来反映一个样本的异常程度。这个数值的大致意思是：一个样本点周围的样本点所处位置的平均密度比上该样本点所在位置的密度。比值越大于1，则该点所在位置的密度越小于其周围样本所在位置的密度，这个点就越有可能是异常点。<br>
如LOF这类基于密度或距离的算法，适用于二维或高维坐标体系内异常点的判别，例如二维平面坐标或经纬度空间坐标下异常点识别。主要是通过比较每个点p和其邻域点的密度来判断该点是否为异常点，如果点p的密度越低，越可能被认定是异常点。至于密度，是通过点之间的距离来计算的，点之间距离越远，密度越低，距离越近，密度越高。

In [15]:
# LOF算法
begin_time = time()
lof_roc = []
lof_rank_n =[]

for i in range(length):
    df = pd.read_csv(wine_bm_files[i])
    # 选取属性特征
    Xattr = df[["fixed.acidity","volatile.acidity","volatile.acidity", "residual.sugar", "chlorides", 
               "free.sulfur.dioxide", "total.sulfur.dioxide", "density", "sulphates", "alcohol"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'LOF'
    clf = LOF()  
    clf.fit(X_train) # 拟合检测器clf
    
    # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        lof_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        lof_rank_n.append(prn_)
    except ValueError:
        pass
   
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)
    
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
wine_roc_mean.append(np.mean(lof_roc))
wine_rank_n_mean.append(np.mean(lof_rank_n))
wine_time_total.append(run_time)

Current Iter 0
On Training Data:
LOF ROC:0.575, precision @ rank n:0.094

On Test Data:
LOF ROC:0.6277, precision @ rank n:0.0862
Current Iter 1000
On Training Data:
LOF ROC:0.575, precision @ rank n:0.426

On Test Data:
LOF ROC:0.5548, precision @ rank n:0.3995


In [16]:
# 可视化展示
x_ = [x for x in range(len(lof_roc))]
trace = go.Scatter(x = x_, y = lof_roc, mode = "lines + markers", name = "LOF_ROC",
                   marker = dict(color='rgba(249, 94, 28, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'LOF_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='LOF_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="2.4"></a> <br>
> ###  **2.4 Method: IForest**

**孤立森林（Isolation Forest)** 是另外一种高效的异常检测算法，它和随机森林类似，但每次选择划分属性和划分点（值）时都是随机的，而不是根据信息增益或者基尼指数来选择。在建树过程中，如果一些样本很快就到达了叶子节点（即叶子到根的距离d很短），那么就被认为很有可能是异常点。因为那些路径d比较短的样本，都是因为距离主要的样本点分布中心比较远的。也就是说，可以通过计算样本在所有树中的平均路径长度来寻找异常点。

In [17]:
# IForest算法
begin_time = time()
iforest_roc = []
iforest_rank_n =[]

for i in range(length):
    df = pd.read_csv(wine_bm_files[i])
    # 选取属性特征
    Xattr = df[["fixed.acidity","volatile.acidity","volatile.acidity", "residual.sugar", "chlorides", 
               "free.sulfur.dioxide", "total.sulfur.dioxide", "density", "sulphates", "alcohol"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'IForest'
    clf = IForest(n_estimators = 50)  
    clf.fit(X_train) # 拟合检测器clf
    
    # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        iforest_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        iforest_rank_n.append(prn_)
    except ValueError:
        pass
   
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)
    
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
wine_roc_mean.append(np.mean(iforest_roc))
wine_rank_n_mean.append(np.mean(iforest_rank_n))
wine_time_total.append(run_time)

Current Iter 0
On Training Data:
IForest ROC:0.5789, precision @ rank n:0.0744

On Test Data:
IForest ROC:0.4869, precision @ rank n:0.0926
Current Iter 1000
On Training Data:
IForest ROC:0.5253, precision @ rank n:0.3891

On Test Data:
IForest ROC:0.5292, precision @ rank n:0.3806


In [18]:
# 可视化展示
x_ = [x for x in range(len(iforest_roc))]
trace = go.Scatter(x = x_, y = iforest_roc, mode = "lines + markers", name = "IForest_ROC",
                   marker = dict(color='rgba(249, 94, 28, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'IForest_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='IForest_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="2.5"></a> <br>
> ###  **2.5 Conclusion Analysis**

下面我们通过ROC均值，Rank_n均值以及算法运行时间来对上述不同算法进行比较：

In [19]:
# roc 和 rank_n值散点图
Roc_ = np.array(wine_roc_mean)
Rank_n = np.array(wine_rank_n_mean)
trace1 = go.Scatter(x = [Rank_n[0]], y = [Roc_[0]], mode = "markers", name = "KNN",
                    marker = dict(color='rgba(28, 149, 249, 0.8)', size=8, symbol=20),
                    text="KNN")
trace2 = go.Scatter( x = [Rank_n[1]], y = [Roc_[1]], mode = "markers", name = "CBLOF",
                    marker = dict(color='rgba(249, 94, 28, 0.8)', size=8, symbol=20),
                    text="CBLOF")
trace3 = go.Scatter(x = [Rank_n[2]], y = [Roc_[2]], mode = "markers", name = "LOF",
                    marker = dict(color='rgba(150, 26, 80, 0.8)', size=8, symbol=20),
                    text= "LOF")
trace4 = go.Scatter(x = [Rank_n[3]], y = [Roc_[3]], mode = "markers", name = "IForest",
                    marker = dict(color='lime', size=8, symbol=20),
                    text = "IForest")                 

data = [trace1, trace2, trace3, trace4]
layout = dict(title = 'Roc & Rank_n Plot',
              xaxis = dict(title='Rank_n', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='Roc', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)',
             )
fig = dict(data=data, layout=layout)
iplot(fig)

In [20]:
# 算法执行时间
Method = ['KNN', 'CBLOF', 'LOF', 'IFprest']

trace = go.Bar(x=Method, y=wine_time_total,
               marker=dict(color=["rgb(1,15,139)", "rgb(49,54,149)", "rgb(69,117,180)", "rgb(171,217,233)"]), 
               opacity = 0.75)
layout = go.Layout(title='Time Cost for Different Method',
                   xaxis=dict(title='Method'), yaxis=dict(title='Time Cost'), bargap=0.2,
                   bargroupgap=0.1, paper_bgcolor='rgb(243, 243, 243)', plot_bgcolor="rgb(243, 243, 243)")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

从上述roc和rank_N值散点图和算法执行时间柱状图可以看出（该图中将鼠标hover到图上可以看到具体数值情况），**LOF算法**的效果最好，同时其算法执行时间是这里采用的四种方法中最短的；另一方面**KNN算法**的检测效果在剩下三者中更好一些，但是其执行时间却是四个方法中最长的，所以在该数据集上采用LOF算法比较好。

## <a id="3"></a> <br>
> ## **3. Skin benchmark**

以skin_benchmark中第一个数据集为例，我们先来看一下数据集中的数据概况：

In [21]:
skinExp = pd.read_csv("./skin/benchmarks/skin_benchmark_0001.csv")

首先让我们先来看看该数据集的头和尾，下表显示了数据集开头的前4行。<br>

In [22]:
d = skinExp.head(5)
colorscale = "YlOrRd"
table = ff.create_table(d, colorscale=colorscale)
for i in range(len(table.layout.annotations)):
    table.layout.annotations[i].font.size = 9
    table.layout.annotations[i].font.color = 'black'
iplot(table)

In [23]:
skinExp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   point.id        6000 non-null   object 
 1   motherset       6000 non-null   object 
 2   origin          6000 non-null   object 
 3   original.label  6000 non-null   int64  
 4   diff.score      6000 non-null   float64
 5   ground.truth    6000 non-null   object 
 6   R               6000 non-null   float64
 7   G               6000 non-null   float64
 8   B               6000 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 422.0+ KB


从上面的例子可以看到，数据集中每一个.csv文件包含多条skin_point的数据记录，同时数据中也给出了skin_point所对应的RGB数值，为了能够有test集合上进行验证的，我们发现GroundTruth于origin.label提供了每一条记录点是否为异常点的记录，于是将此作为test集合的label。下面我们来分别处理benchmarks中的每一个.csv文件：

In [24]:
skin_bm_path = "./skin/benchmarks"
skin_bm_files = []
for skin_bm_file in os.listdir(skin_bm_path):
    file_path = os.path.join(skin_bm_path, skin_bm_file)
    skin_bm_files.append(file_path)
    
# 打乱benchmarks中的.csv文件顺序    
np.random.shuffle(skin_bm_files)
# 后面算法用来分割训练集与测试集
train_rate = 0.7

length = len(skin_bm_files)
print("=" * 70)
print("The Total Number of Benchmark is: " , length)
print("e.g. : ", skin_bm_files[0])
print("=" * 70)

The Total Number of Benchmark is:  1500
e.g. :  ./skin/benchmarks/skin_benchmark_1084.csv


In [25]:
df = pd.read_csv(skin_bm_files[0])
print("=" * 70)
print("The shape of DataFrame is:", df.shape)
print("=" * 70)

d = df.head()
colorscale = "YlOrRd"
table = ff.create_table(d, colorscale=colorscale)
for i in range(len(table.layout.annotations)):
    table.layout.annotations[i].width = 50
    table.layout.annotations[i].font.size = 9
    table.layout.annotations[i].font.color = 'black'
iplot(table)

The shape of DataFrame is: (6000, 9)


读取数据集中的数据后可以看到，该数据集总共包括1500个benchmark，且其中每个benchmark中有6000条记录，在这里我们选取其中的**R** & **G** & **B**三个属性值，同时选取**ground.truth**值来完成本次离群点分析和异常检测的实验。<br>
其中分析算法我们选择：**KNN** & **CBLOF** & **LOF** & **IForest** 

<a id="3.1"></a> <br>
> ###  **3.1 Method: KNN**

基于**k-近邻平均距离（kNN）的离群点检测算法**是一种比较简单的检测方法，其算法的流程是：输入数据集D，参数k和n；对于每个点计算它的k-近邻距离；按照k-近邻距离进行降序排序，前n个点即可认为是离群点，代码如下所示：

In [26]:
skin_roc_mean = []
skin_rank_n_mean = []
skin_time_total = []

# KNN算法
begin_time = time()
knn_roc = []
knn_rank_n =[]

for i in range(length):
    df = pd.read_csv(skin_bm_files[i])
    # 选取属性特征
    Xattr = df[["R","G","B"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'KNN'
    clf = KNN(n_neighbors = 5) 
    clf.fit(X_train) # 拟合检测器clf
    
    # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        knn_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        knn_rank_n.append(prn_)
    except ValueError:
        pass
   
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)
    
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
skin_roc_mean.append(np.mean(knn_roc))
skin_rank_n_mean.append(np.mean(knn_rank_n))
skin_time_total.append(run_time)

Current Iter 0
On Training Data:
KNN ROC:0.9573, precision @ rank n:0.0256

On Test Data:
KNN ROC:0.9506, precision @ rank n:0.0
Current Iter 1000
On Training Data:
KNN ROC:0.7781, precision @ rank n:0.0815

On Test Data:
KNN ROC:0.7805, precision @ rank n:0.1487


In [27]:
# 可视化展示
x_ = [x for x in range(len(knn_roc))]
trace = go.Scatter(x = x_, y = knn_roc, mode = "lines + markers", name = "KNN_ROC",
                   marker = dict(color='rgba(150, 26, 80, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'KNN_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='KNN_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="3.2"></a> <br>
> ###  **3.2 Method: CBLOF**

**CBLOF算法**是基于聚类的方法，基于聚类的方法通过考察对象与簇之间的关系检测离群点。其算法的流程是：找出数据集中的簇，并把它们按大小降序排列；对于每个数据点赋予基于簇的局部离群点因子（CBLOF），对于属于大簇的点，它的CBLOF是簇的大小和该点与簇的相似性的乘积。CBLOF用统计学方法定义点和簇之间的相似性，代表点属于簇的概率，该值越大，点与簇越相似，CBLOF值可以检测远离任何簇的离群点。

In [28]:
# CBLOF算法
begin_time = time()
cblof_roc = []
cblof_rank_n = []

for i in range(length):
    df = pd.read_csv(skin_bm_files[i])
    # 选取属性特征
    Xattr = df[["R","G","B"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'CBLOF'
    clf = CBLOF(random_state=42) 
    clf.fit(X_train) # 拟合检测器clf
    
     # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        cblof_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        cblof_rank_n.append(prn_)
    except ValueError:
        pass
    
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)

        
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
skin_roc_mean.append(np.mean(cblof_roc))
skin_rank_n_mean.append(np.mean(cblof_rank_n))
skin_time_total.append(run_time)

Current Iter 0
On Training Data:
CBLOF ROC:0.7929, precision @ rank n:0.0

On Test Data:
CBLOF ROC:0.7781, precision @ rank n:0.0
Current Iter 1000
On Training Data:
CBLOF ROC:0.8787, precision @ rank n:0.5189

On Test Data:
CBLOF ROC:0.852, precision @ rank n:0.4261


In [29]:
# 可视化展示
x_ = [x for x in range(len(cblof_roc))]
trace = go.Scatter(x = x_, y = cblof_roc, mode = "lines + markers", name = "CBLOF_ROC",
                   marker = dict(color='rgba(150, 26, 80, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'CBLOF_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='CBLOF_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="3.3"></a> <br>
> ###  **3.3 Method: LOF**

**LOF算法**是一种基于距离的异常检测方法，LOF通过计算一个数值score来反映一个样本的异常程度。这个数值的大致意思是：一个样本点周围的样本点所处位置的平均密度比上该样本点所在位置的密度。比值越大于1，则该点所在位置的密度越小于其周围样本所在位置的密度，这个点就越有可能是异常点。<br>
如LOF这类基于密度或距离的算法，适用于二维或高维坐标体系内异常点的判别，例如二维平面坐标或经纬度空间坐标下异常点识别。主要是通过比较每个点p和其邻域点的密度来判断该点是否为异常点，如果点p的密度越低，越可能被认定是异常点。至于密度，是通过点之间的距离来计算的，点之间距离越远，密度越低，距离越近，密度越高。

In [30]:
# LOF算法
begin_time = time()
lof_roc = []
lof_rank_n =[]

for i in range(length):
    df = pd.read_csv(skin_bm_files[i])
    # 选取属性特征
    Xattr = df[["R","G","B"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'LOF'
    clf = LOF()  
    clf.fit(X_train) # 拟合检测器clf
    
    # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        lof_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        lof_rank_n.append(prn_)
    except ValueError:
        pass
   
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)
    
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
skin_roc_mean.append(np.mean(lof_roc))
skin_rank_n_mean.append(np.mean(lof_rank_n))
skin_time_total.append(run_time)

Current Iter 0
On Training Data:
LOF ROC:0.7436, precision @ rank n:0.0

On Test Data:
LOF ROC:0.8073, precision @ rank n:0.0476
Current Iter 1000
On Training Data:
LOF ROC:0.4489, precision @ rank n:0.0187

On Test Data:
LOF ROC:0.4479, precision @ rank n:0.0


In [31]:
# 可视化展示
x_ = [x for x in range(len(lof_roc))]
trace = go.Scatter(x = x_, y = lof_roc, mode = "lines + markers", name = "LOF_ROC",
                   marker = dict(color='rgba(150, 26, 80, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'LOF_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='LOF_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="3.4"></a> <br>
> ###  **3.4 Method: IForest**

**孤立森林（Isolation Forest)** 是另外一种高效的异常检测算法，它和随机森林类似，但每次选择划分属性和划分点（值）时都是随机的，而不是根据信息增益或者基尼指数来选择。在建树过程中，如果一些样本很快就到达了叶子节点（即叶子到根的距离d很短），那么就被认为很有可能是异常点。因为那些路径d比较短的样本，都是因为距离主要的样本点分布中心比较远的。也就是说，可以通过计算样本在所有树中的平均路径长度来寻找异常点。

In [32]:
# IForest算法
begin_time = time()
iforest_roc = []
iforest_rank_n =[]

for i in range(length):
    df = pd.read_csv(skin_bm_files[i])
    # 选取属性特征
    Xattr = df[["R","G","B"]].values
    labels = np.where(df["ground.truth"] == "nominal", 0, 1)

    Xattr,labels = shuffle(Xattr,labels)
    X_train = Xattr[:int(train_rate*len(Xattr))]
    X_test = Xattr[int(train_rate*len(Xattr)):]
    y_train = labels[:int(train_rate*len(Xattr))]
    y_test = labels[int(train_rate*len(Xattr)):]
    if np.sum(y_test) < 5:
        continue
    
    clf_name = 'IForest'
    clf = IForest(n_estimators = 50)  
    clf.fit(X_train) # 拟合检测器clf
    
    # 给出预测标签和训练数据的离群点得分
    y_train_pred = clf.labels_  # 二元数据 (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  

    # 在测试数据上进行预测
    y_test_pred = clf.predict(X_test) # 离群点标签0或1
    y_test_scores = clf.decision_function(X_test) 
    
    y = column_or_1d(y_test)
    y_pred = column_or_1d(y_test_scores)
    #check_consistent_length(y, y_pred)
    
    try:
        roc_ = np.round(roc_auc_score(y, y_pred), decimals=4)
        iforest_roc.append(roc_)
        prn_ = np.round(precision_n_scores(y, y_pred), decimals=4)
        iforest_rank_n.append(prn_)
    except ValueError:
        pass
   
    if i % 1000 == 0:
        # 输出结果
        print("=" * 70)
        print("Current Iter " + str(i))
        print("=" * 70)
        print("On Training Data:")
        evaluate_print(clf_name, y_train, y_train_scores)
        print("\nOn Test Data:")
        evaluate_print(clf_name, y_test, y_test_scores)
    
# 记录每个算法运行时间   
end_time = time()
run_time = end_time - begin_time
skin_roc_mean.append(np.mean(iforest_roc))
skin_rank_n_mean.append(np.mean(iforest_rank_n))
skin_time_total.append(run_time)

Current Iter 0
On Training Data:
IForest ROC:0.467, precision @ rank n:0.0

On Test Data:
IForest ROC:0.4764, precision @ rank n:0.0
Current Iter 1000
On Training Data:
IForest ROC:0.8998, precision @ rank n:0.2616

On Test Data:
IForest ROC:0.9038, precision @ rank n:0.3037


In [33]:
# 可视化展示
x_ = [x for x in range(len(iforest_roc))]
trace = go.Scatter(x = x_, y = iforest_roc, mode = "lines + markers", name = "IForest_ROC",
                   marker = dict(color='rgba(150, 26, 80, 0.8)', size=6),
                  )
data = [trace]
layout = dict(title = 'IForest_Roc Line & Scatter Plot',
              xaxis = dict(title='Number', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='IForest_ROC', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)' )
fig = dict(data=data, layout=layout)
iplot(fig)

<a id="3.5"></a> <br>
> ###  **3.5 Conclusion Analysis**

下面我们通过ROC均值，Rank_n均值以及算法运行时间来对上述不同算法进行比较：

In [35]:
# roc 和 rank_n值散点图
Roc_ = np.array(skin_roc_mean)
Rank_n = np.array(skin_rank_n_mean)
trace1 = go.Scatter(x = [Rank_n[0]], y = [Roc_[0]], mode = "markers", name = "KNN",
                    marker = dict(color='rgba(28, 149, 249, 0.8)', size=8, symbol=20),
                    text="KNN")
trace2 = go.Scatter( x = [Rank_n[1]], y = [Roc_[1]], mode = "markers", name = "CBLOF",
                    marker = dict(color='rgba(249, 94, 28, 0.8)', size=8, symbol=20),
                    text="CBLOF")
trace3 = go.Scatter(x = [Rank_n[2]], y = [Roc_[2]], mode = "markers", name = "LOF",
                    marker = dict(color='rgba(150, 26, 80, 0.8)', size=8, symbol=20),
                    text= "LOF")
trace4 = go.Scatter(x = [Rank_n[3]], y = [Roc_[3]], mode = "markers", name = "IForest",
                    marker = dict(color='lime', size=8, symbol=20),
                    text = "IForest")                 

data = [trace1, trace2, trace3, trace4]
layout = dict(title = 'Roc & Rank_n Plot',
              xaxis = dict(title='Rank_n', ticklen=5, zeroline=False, zerolinewidth=1, gridcolor="white"),
              yaxis = dict(title='Roc', ticklen= 5, zeroline= False, zerolinewidth=1, gridcolor="white",),
              paper_bgcolor='rgb(243, 243, 243)',
              plot_bgcolor='rgb(243, 243, 243)',
             )
fig = dict(data=data, layout=layout)
iplot(fig)

In [37]:
# 算法执行时间
Method = ['KNN', 'CBLOF', 'LOF', 'IForest']

trace = go.Bar(x=Method, y=skin_time_total,
               marker=dict(color=["rgb(1,15,139)", "rgb(49,54,149)", "rgb(69,117,180)", "rgb(171,217,233)"]), 
               opacity = 0.75)
layout = go.Layout(title='Time Cost for Different Method',
                   xaxis=dict(title='Method'), yaxis=dict(title='Time Cost'), bargap=0.2,
                   bargroupgap=0.1, paper_bgcolor='rgb(243, 243, 243)', plot_bgcolor="rgb(243, 243, 243)")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

从上述roc和rank_N值散点图和算法执行时间柱状图可以看出（该图中将鼠标hover到图上可以看到具体数值情况），**KNN算法**的效果最好，但是其算法执行时间是这里采用的四种方法中最长的；**CBLOF算法**的检测效果与KNN算法相差不大，同时其执行时间在四个方法中排第二，所以在该数据集上采用CBLOF算法比较好；另一方面虽然**LOF算法**在四种算法中耗时最短，但其检测效果最差。所以在选择不同算法时需要综合考虑各种因素。